# Reduce ImageCollection

In [ ]:
import ee, geetools, os, re, httplib2

if "EARTHENGINE_SERVICE_ACCOUNT" in os.environ:
    private_key = os.environ["EARTHENGINE_SERVICE_ACCOUNT"]
    private_key = private_key[1:-1] if re.compile(r"^'[^']*'$").match(private_key) else private_key
    ee.Initialize.geetools.from_service_account(private_key)

elif "EARTHENGINE_PROJECT" in os.environ:
    ee.Initialize(project=os.environ["EARTHENGINE_PROJECT"], http_transport=httplib2.Http())

else:
    raise ValueError("EARTHENGINE_SERVICE_ACCOUNT or EARTHENGINE_PROJECT environment variable is missing")

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geetools/blob/main/docs/usage/reduce.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20colab-blue?logo=googlecolab&labelColor=555)](https://colab.research.google.com/github/gee-community/geetools/blob/main/docs/usage/reduce.ipynb)

THe Earth Engine API provides 2 ways to reduce images: `reduceRegion` and `reduceRegion`. `geetools` is making these methods also available for `ee.ImageCollection` objects.

## Set up environment

Install all the required libs if necessary and perform the import statements upstream.

In [ ]:
# uncomment if installation of libs is necessary
# !pip install earthengine-api geetools

In [ ]:
import ee
import geetools #noqa: F401

In [ ]:
# uncomment if initialization is required
# ee.Initialize()

## Example data 

The following examples rely on a `ee.FeatureCollection` composed of three ecoregion features that define regions by which to reduce image data. The ImageCollection data loads the modis vegetation indicies and subset the 2010 2020 decade of images.

In [ ]:
## Import the example feature collection and drop the data property.
ecoregions = (
    ee.FeatureCollection("projects/google/charts_feature_example")
    .select(["label", "value", "warm"])
)


## Load MODIS vegetation indices data and subset of 4 images.
vegIndices = (
    ee.ImageCollection("MODIS/061/MOD13A1")
    .filter(ee.Filter.date("2010-01-01", "2010-02-28"))
    .select(["NDVI", "EVI"])
)

## Reduce over single region

Using `reduceRegion` you can reduce an `ee.ImageCollection` over a single region. The function will return a `ee.Dictionary` with the reduced values of each band grouped under each image Id as key.

It will return a `ee.Dictionary` with the following shape: 

```json
{
    "image1": {"band1": value1, "band2": value2, ...},
    "image2": {"band1": value1, "band2": value2, ...},
}
```

where `image*`is the id of the image as per specified property (casted to string) and `band*` is the name of the band.

In [ ]:
vegIndices.geetools.reduceRegion(
    reducer = ee.Reducer.mean(),
    idProperty = "system:time_start",
    idType = ee.Date,
    geometry = ecoregions.filter(ee.Filter.eq("label", "Forest")).geometry(),
    scale = 500
).getInfo()